In [19]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

legislatura = 'Legislatura XLVIII'


### Acerca del parlamento

In [20]:
evolucion_proyectos_df = pd.read_csv('data/%s/evolucion_proyectos.csv' % legislatura, index_col=[1, 0])
evolucion_proyectos_df.head()

fecha cuerpo       estado  tipo_ref  ref_id
id_ficha                                                    
138855   0  2018-03-13    CSS      ENTRADA       NaN     NaN
         1  2018-03-14    CSS      ENTRADA  COMISION   927.0
         2  2019-04-08    CSS  TRATAMIENTO  COMISION   927.0
125838   0  2015-08-05    CSS      ENTRADA       NaN     NaN
         1  2015-08-05    CSS      ENTRADA  COMISION   331.0

In [21]:
proyectos_df = pd.read_csv('data/%s/proyectos.csv' % legislatura, index_col=0)
proyectos_df.head()

,id_ficha,fecha_entrada,tipo,titulo,origen,presentado_por
0,133188,2016-12-14,RESOLUCION,"AGAZZI, Ernesto. SENADOR REPUBLICA. RENUNCIA.",Cámara Senadores,"Agazzi, Ernesto"
1,132804,2016-11-22,PROYECTO DE LEY,ENTES AUTÓNOMOS. SERVICIOS DESCENTRALIZADOS. G...,Cámara Senadores,"Agazzi, Ernesto; Ayala, Patricia; Berterreche,..."
2,132477,2016-11-01,DECLARACION,REPÚBLICA ITALIANA. TERREMOTOS. SOLIDARIDAD. M...,Cámara Senadores,"Agazzi, Ernesto; Amorín, José; Delgado, Alvaro..."
3,132117,2016-10-18,LEY,INGENIERO RICARDO ROSA. TRAMO RUTA 67 (DESDE E...,Cámara Senadores,"Agazzi, Ernesto; Ayala, Patricia; Carámbula Vo..."
4,132114,2016-10-18,LEY,"TOSCANINI, JUAN. TRAMO RUTA 69 (DESDE INTERSEC...",Cámara Senadores,"Agazzi, Ernesto; Ayala, Patricia; Carámbula Vo..."


### Acerca de los parlamentarios (Senadores)

In [23]:
asistencia_df = pd.read_csv('data/%s/senado_asistencia.csv' % legislatura, index_col=0)
trabajo_df = pd.read_csv('data/%s/senado_trabajo.csv' % legislatura, index_col=0)

tipo_tareas_df = trabajo_df[['tipo']].drop_duplicates()

# Cálculo de asistencias
asistencia_df.loc[:,'asistencia_comision'] = asistencia_df.apply(lambda x : x['citacion_comision'] - x['falta_con_aviso_comision'] - x['falta_sin_aviso_comision'] - x['licencia_comision'], axis=1)
asistencia_df.loc[:,'citaciones'] = asistencia_df.apply(lambda x : x['citado_camara'] + x['citacion_comision'] , axis=1)
asistencia_df.loc[:,'asistencia'] = asistencia_df.apply(lambda x : x['asistencia_camara'] + x['asistencia_comision'], axis=1)
asistencia_df.loc[:,'con_licencia'] = asistencia_df.apply(lambda x : x['con_licencia_camara'] + x['licencia_comision'], axis=1)
asistencia_df.loc[:,'faltas_sin_aviso'] = asistencia_df.apply(lambda x : x['faltas_sin_aviso_camara'] + x['falta_sin_aviso_comision'], axis=1)
asistencia_df.loc[:,'asistencia_nota'] = asistencia_df.apply(lambda x : x['asistencia'] * 1.0 / (x['citaciones'] - x['con_licencia']) * 100 , axis=1)
asistencia_df.loc[:,'faltas_sin_aviso_nota'] = asistencia_df.apply(lambda x : 100 - (x['faltas_sin_aviso'] / (x['citaciones'] - x['con_licencia']) * 100), axis=1)

# Cálculo de cantidad de tareas realizadas
senadores_trabajo_df = trabajo_df[['id_senador', 'id_ficha']].groupby(['id_senador']).count().reset_index(level=0)
senadores_trabajo_df = senadores_trabajo_df.merge(trabajo_df[['id_senador', 'nombre']].drop_duplicates(subset=['id_senador']).reset_index(drop=True), on=['id_senador'], how='left')
senadores_trabajo_df = senadores_trabajo_df.rename({'id_ficha': 'cant_tareas'}, axis=1)

# Cantidad de pedidos de informe
senadores_por_tipo_df = trabajo_df[trabajo_df['tipo'] == 'PEDIDO DE INFORME'][['id_senador', 'id_ficha']].groupby(['id_senador']).count().reset_index(level=0)
senadores_trabajo_df = senadores_trabajo_df.merge(senadores_por_tipo_df[['id_senador', 'id_ficha']].drop_duplicates(subset=['id_senador']).reset_index(drop=True), on=['id_senador'], how='left')
senadores_trabajo_df = senadores_trabajo_df.rename({'id_ficha': 'cant_pedido_informe'}, axis=1)

# Cantidad de otro tipo de trabajos (RESOLUCIONES, HOMENAJES, etc)
senadores_por_tipo_df = trabajo_df[(trabajo_df['tipo'] != 'PEDIDO_INFORME') & (trabajo_df['tipo'] != 'LEY') & (trabajo_df['tipo'] != 'PROYECTO DE LEY')][['id_senador', 'id_ficha']].groupby(['id_senador']).count().reset_index(level=0)
senadores_trabajo_df = senadores_trabajo_df.merge(senadores_por_tipo_df[['id_senador', 'id_ficha']].drop_duplicates(subset=['id_senador']).reset_index(drop=True), on=['id_senador'], how='left')
senadores_trabajo_df = senadores_trabajo_df.rename({'id_ficha': 'cant_otros_tipos'}, axis=1)

# Cantidad de leyes y proyectos de leyes presentados
senadores_por_tipo_df = trabajo_df[(trabajo_df['tipo'] == 'LEY') | (trabajo_df['tipo'] == 'PROYECTO DE LEY')][['id_senador', 'id_ficha']].groupby(['id_senador']).count().reset_index(level=0)
senadores_trabajo_df = senadores_trabajo_df.merge(senadores_por_tipo_df[['id_senador', 'id_ficha']].drop_duplicates(subset=['id_senador']).reset_index(drop=True), on=['id_senador'], how='left')
senadores_trabajo_df = senadores_trabajo_df.rename({'id_ficha': 'cant_leyes'}, axis=1)

# Cantidad de leyes y proyecto de leyes aprobados (promulgados por el ejecutivo)
senadores_por_tipo_df = trabajo_df[(trabajo_df['estado'] == 'PROMULGA')][['id_senador', 'id_ficha']].groupby(['id_senador']).count().reset_index(level=0)
senadores_trabajo_df = senadores_trabajo_df.merge(senadores_por_tipo_df[['id_senador', 'id_ficha']].drop_duplicates(subset=['id_senador']).reset_index(drop=True), on=['id_senador'], how='left')
senadores_trabajo_df = senadores_trabajo_df.rename({'id_ficha': 'cant_leyes_aprobado'}, axis=1)

senadores_df = asistencia_df.merge(senadores_trabajo_df.drop(columns=['nombre']).reset_index(drop=True), on=['id_senador'], how='left')

# Cálculo de la nota en tareas presentadas
max_data = senadores_df[['cant_tareas']].max()[0]
senadores_df.loc[:,'cant_tareas_nota'] = senadores_df['cant_tareas'] / max_data * 100.0

# Cálculo de la nota en pedidos de informe
max_data = senadores_df[['cant_pedido_informe']].max()[0]
senadores_df.loc[:,'cant_pedido_informe_nota'] = senadores_df['cant_pedido_informe'] / max_data * 100.0

# Cálculo de la nota en otro tipo de documentos
max_data = senadores_df[['cant_otros_tipos']].max()[0]
senadores_df.loc[:,'cant_otros_tipos_nota'] = senadores_df['cant_otros_tipos'] / max_data * 100.0

# Cálculo de la nota en leyes presentadas
max_data = senadores_df[['cant_leyes']].max()[0]
senadores_df.loc[:,'cant_leyes_nota'] = senadores_df['cant_leyes'] / max_data * 100.0

# Cálculo de la nota en tareas presentadas y aprobadas 
max_data = senadores_df[['cant_leyes_aprobado']].max()[0]
senadores_df.loc[:,'cant_leyes_aprobado_nota'] = senadores_df['cant_leyes_aprobado'] / max_data * 100.0

# Cálculo de nota general
w_asistencia_nota=1.0
w_faltas_sin_aviso_nota=1.0
w_cant_tareas_nota=1.0
w_cant_pedido_informe_nota=1.0
w_cant_otros_tipos_nota=1.0
w_cant_leyes_nota=1.0
w_cant_leyes_aprobado_nota=1.0

senadores_df = senadores_df.apply(lambda x : x.fillna('') if x.name == 'email' else x.fillna(0))
senadores_df.loc[:,'nota_general'] = (senadores_df['asistencia_nota'] * w_asistencia_nota + \
    senadores_df['faltas_sin_aviso_nota']  * w_faltas_sin_aviso_nota + \
    senadores_df['cant_tareas_nota'] * w_cant_tareas_nota + \
    senadores_df['cant_pedido_informe_nota'] * w_cant_pedido_informe_nota + \
    senadores_df['cant_otros_tipos_nota'] * w_cant_otros_tipos_nota + \
    senadores_df['cant_leyes_nota'] * w_cant_leyes_nota + \
    senadores_df['cant_leyes_aprobado_nota'] * w_cant_leyes_aprobado_nota) / 7

del asistencia_df, trabajo_df, senadores_por_tipo_df, senadores_trabajo_df
senadores_df.head()

,id_senador,nombre,lema,email,citado_camara,asistencia_camara,falta_con_aviso_camara,faltas_sin_aviso_camara,con_licencia_camara,pasaje_presidencia_camara,citacion_comision,falta_con_aviso_comision,falta_sin_aviso_comision,licencia_comision,otras_comisiones_comision,asistencia_comision,citaciones,asistencia,con_licencia,faltas_sin_aviso,asistencia_nota,faltas_sin_aviso_nota,cant_tareas,cant_pedido_informe,cant_otros_tipos,cant_leyes,cant_leyes_aprobado,cant_tareas_nota,cant_pedido_informe_nota,cant_otros_tipos_nota,cant_leyes_nota,cant_leyes_aprobado_nota,nota_general
0,5146,"Agazzi, Ernesto",0,eagazzi@parlamento.gub.uy,108,98,0,0,10,0,131.0,8.0,0.0,0.0,1.0,123.0,239.0,221.0,10.0,0.0,96.506550,100.0,262.0,1.0,87.0,175.0,0.0,51.881188,2.127660,45.312500,39.954338,0.0,47.968891
1,2743,"Alcorta, Ricardo",PARTIDO FRENTE AMPLIO,,25,25,0,0,0,0,53.0,5.0,0.0,0.0,1.0,48.0,78.0,73.0,0.0,0.0,93.589744,100.0,3.0,0.0,3.0,0.0,0.0,0.594059,0.000000,1.562500,0.000000,0.0,27.963758
2,8883,"Almagro Lemes, Luis Leonardo",0,secretaria.ministro@mrree.gub.uy,13,4,0,0,9,0,12.0,7.0,0.0,0.0,0.0,5.0,25.0,9.0,9.0,0.0,56.250000,100.0,1.0,0.0,1.0,0.0,0.0,0.198020,0.000000,0.520833,0.000000,0.0,22.424122
3,9891,"Alonso, Verónica",PARTIDO NACIONAL,valonso@parlamento.gub.uy,227,134,21,0,72,0,352.0,131.0,0.0,1.0,0.0,220.0,579.0,354.0,73.0,0.0,69.960474,100.0,183.0,30.0,108.0,75.0,0.0,36.237624,63.829787,56.250000,17.123288,0.0,49.057310
4,11535,"Álvarez López, María Dolores",PARTIDO NACIONAL,nolivera@parlamento.gub.uy,6,5,1,0,0,0,19.0,0.0,10.0,0.0,0.0,9.0,25.0,14.0,0.0,10.0,56.000000,60.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,16.571429


### Lista de notas más bajas en asistencia a cámara y comisiones (100 = sin faltas)

In [24]:
senadores_df[['nombre', 'asistencia', 'asistencia_nota']][senadores_df['asistencia_nota'] > 0].sort_values(['asistencia_nota'], ascending=True).head(n=10)

,nombre,asistencia,asistencia_nota
70,"Montaner, Martha",54.0,54.000000
26,"Cardoso, José Carlos",199.0,55.586592
4,"Álvarez López, María Dolores",14.0,56.000000
2,"Almagro Lemes, Luis Leonardo",9.0,56.250000
82,"Peña Fernández, Daniel",111.0,65.294118
65,"Martínez Villamil, Daniel Carlos",4.0,66.666667
40,"Falero, José Luis",2.0,66.666667
3,"Alonso, Verónica",354.0,69.960474
75,"Olano Rodríguez, Celina Walkiria",14.0,70.000000
53,"Iturralde Viñas, Pablo",139.0,72.020725


### Lista de tipos de tareas parlamentarias que se consideran

In [25]:
tipo_tareas_df

,tipo
0,PEDIDO DE INFORME
835,PROYECTO DE LEY
890,RESOLUCION
901,LEY
929,MINUTA DE COMUNICACION
1152,PROYECTO DE DECLARACION
1554,PROYECTO DE RESOLUCION
2194,PROYECTO DE RESOLUCION
2311,DECLARACION
2327,RESOLUCION


### Lista de notas más bajas en faltas sin aviso a cámara y comisiones (100 = sin faltas sin aviso)

In [26]:
senadores_df[['nombre', 'faltas_sin_aviso', 'faltas_sin_aviso_nota']][senadores_df['faltas_sin_aviso_nota'] > 0].sort_values(['faltas_sin_aviso_nota'], ascending=True).head(n=10)

,nombre,faltas_sin_aviso,faltas_sin_aviso_nota
4,"Álvarez López, María Dolores",10.0,60.000000
51,"Iglesias Rodríguez, Alberto Walter",1.0,88.888889
57,"Lamorte, Aldo",1.0,93.333333
73,"Mujica, Gonzalo",10.0,93.975904
37,"Eguiluz, Cecilia",11.0,94.786730
66,"Matiaude, Graciela",8.0,95.876289
86,"Piñeyrúa Olmos, Ana Lía",1.0,97.560976
25,"Carballo Da Costa, Felipe",3.0,98.739496
29,"Castaingdebat, Armando",2.0,98.843931
53,"Iturralde Viñas, Pablo",2.0,98.963731


### Lista de notas en tareas realizadas (100 = máximo de tareas realizadas)
##### Tareas = pedido de informe, leyes presentadas, resoluciones presentadas, etc

In [27]:
senadores_df[['nombre', 'cant_tareas', 'cant_tareas_nota']][senadores_df['cant_tareas_nota'] > 0].sort_values(['cant_tareas_nota'], ascending=True).head(n=10)

,nombre,cant_tareas,cant_tareas_nota
2,"Almagro Lemes, Luis Leonardo",1.0,0.198020
31,"Cersósimo, Gustavo",1.0,0.198020
41,"Fernández Farias, Eduardo",3.0,0.594059
1,"Alcorta, Ricardo",3.0,0.594059
63,"Lústemberg Haro, María Cristina",5.0,0.990099
91,"Ramos, Conrado",5.0,0.990099
75,"Olano Rodríguez, Celina Walkiria",6.0,1.188119
71,"Moreira, Carlos",7.0,1.386139
93,"Ribero, María Sara",9.0,1.782178
22,"Botana, Sergio",9.0,1.782178


### Lista de notas generales (100 = máximo nota)

In [28]:
senadores_df[['nombre', 'nota_general']][senadores_df['nota_general'] > 0].sort_values(['nota_general'], ascending=True).head(n=20)

,nombre,nota_general
95,"Sendic, Raúl",2.053866
4,"Álvarez López, María Dolores",16.571429
2,"Almagro Lemes, Luis Leonardo",22.424122
40,"Falero, José Luis",23.809524
65,"Martínez Villamil, Daniel Carlos",23.809524
51,"Iglesias Rodríguez, Alberto Walter",25.396825
6,"Amy, José A.",26.190476
71,"Moreira, Carlos",26.219981
75,"Olano Rodríguez, Celina Walkiria",26.725582
76,"Olesker, Daniel",26.785714
